In [1]:
!pip install datasets -qq
!pip install transformers -qq
!pip install rouge_score evaluate nltk -qq
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

In [3]:
from datasets import load_dataset
import datasets
import evaluate
import nltk
import torch
import pandas as pd 
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from nltk.tokenize import word_tokenize

In [4]:
import tensorflow as tf

In [5]:
from transformers import MvpForConditionalGeneration, MvpTokenizerFast

In [6]:
model_checkpoint = "t5-small"

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer

tokenizer = MvpTokenizerFast.from_pretrained("RUCAIBox/mvp")
model = MvpForConditionalGeneration.from_pretrained("RUCAIBox/mvp-summarization")

In [7]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

# Load the XSum dataset
raw_datasets = load_dataset('xsum', split="train[:100]")
raw_datasets = raw_datasets.train_test_split(test_size=0.1)
raw_datasets
# raw_datasets["train"][0]

<ipython-input-7-d33476f077a5>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 90
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 10
    })
})

In [8]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"])

In [10]:
# Tokenize the input and output sequences
def batch_tokenize_preprocess(batch, tokenizer):
    source, target = batch["document"], batch["summary"]
        
    source_tokenized = tokenizer(
        source, truncation=True
    )
    target_tokenized = tokenizer(
        target, truncation=True
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

In [11]:
def tokenize_function(examples):
    return tokenizer(
        examples["summary"],
        examples["document"],
        truncation=True,
        max_length=1024,
        padding="max_length",
        return_tensors="pt",
    )

In [12]:
# Apply the tokenization function to the dataset
tokenized_dataset = raw_datasets.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer
    ),
    batched=True,
    remove_columns=raw_datasets['train'].column_names
)

tokenized_dataset

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 90
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})

In [13]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [14]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MvpForConditionalGeneration(
  (model): MvpModel(
    (shared): Embedding(50267, 1024, padding_idx=1)
    (encoder): MvpEncoder(
      (embed_tokens): Embedding(50267, 1024, padding_idx=1)
      (embed_positions): MvpLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MvpEncoderLayer(
          (self_attn): MvpAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

In [17]:
batch_size = 1 # Updated batch size
training_args = Seq2SeqTrainingArguments(
    output_dir="res",
    num_train_epochs=10, # Updated number of training epochs
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2, # Updated gradient accumulation steps
    warmup_steps=100, # Updated warmup steps
    weight_decay=0.1,
    evaluation_strategy='steps',
    eval_steps=100, # Updated evaluation steps
    label_smoothing_factor=0.2, # Updated label smoothing factor
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3, # Updated save total limit
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

In [18]:
trainer_output = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MvpTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,4.695500,4.805903
200,3.623600,4.808611
300,3.159100,4.802355
400,2.981700,4.789385


In [19]:
trainer_evaluate = trainer.evaluate()

In [20]:
def summarize_story(prompt, model, tokenizer, max_length=512, num_return_sequences=5):
    inputs = tokenizer(prompt, truncation=True, return_tensors="pt")
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=6,
        early_stopping=True,
        no_repeat_ngram_size=2,
        num_return_sequences=num_return_sequences,
    )

    generated_stories = []
    for output in outputs:
        generated_story = tokenizer.decode(output, skip_special_tokens=True)
        generated_stories.append(generated_story)

    return "\n".join(generated_stories)

In [21]:
prompt = "My mother is a person I admire most. She devoted a lot of time and energy to the upbringing of my two brothers and 1. Despite working hard, she always made time to teach us many useful things which are necessary and important in our later lives. Moreover, she is a good role model for me to follow. She always tries to get on well with people who live next door and help everyone when they are in difficulties, so most of them respect and love her. I admire and look up to my mother because she not only brings me up well but also stands by me and gives some help if necessary. For example, when I encounter some difficulties, she will give me some precious advice to help me solve those problems. She has a major influence on me and 1 hope that I will inherit some of her traits."
stories = summarize_story(prompt, model, tokenizer, num_return_sequences=5)
print(stories)

My mother has a major influence on me and I hope that I will inherit some of her traits.
My mother has a major influence on me and I hope I will inherit some of her traits.
My mother has a major influence on me and I hope to inherit some of her traits.
My mother has a major influence on me.
My mother has been a role model for me.


In [22]:
prompt = "There are five members in my family. They are my father, my mother, my two younger sisters, and me. My father is a doctor. He is tall and kind. He likes playing football very much. He teaches me to play football every day after school. My mother is a housewife. She is short and thin. She cooks so well that everyone in the family loves her food very much. She teaches me how to cook delicious food too. I have two younger sisters - Lan and Hoa. Lan is 12 years old and Hoa is 10 years old. They are good students at school. I love my family."
stories = summarize_story(prompt, model, tokenizer, num_return_sequences=5)
print(stories)

I love my family.
I love my family very much.
I have a family of five.
I have a family that loves football very much.
I have a family of 5.


In [23]:
prompt = "Amidst the immense mountains and forests of Lam Dong province, Dalat, a beautiful town, captivates all those who have been there once. I am one of those. Dalat is often called the City of Eternal Spring. Flowers of all colors, with many species, the most numerous of which are orchids. More than anywhere else in Viet- nam, Dalat sees flowers vie with one another in blos soming in spring. I used to get up early in the morn- ing on fine days to welcome dawn on the highlands. Opening the windows, I had a breath-taking view of nature, and enjoyed the fragrance of wild flowers car- ried by the clouds, I felt relieved in my heart. In the late afternoons, I often reserved for visits to the Valley of Love and Sigh Lake covered with quiet pine forests. Twilight on Dalat also brought many pictures and sen- sations. The wind rustling through the pine forests, the roar of waterfalls, the chirping of birds and the clatter of horse ‘s hooves….all of these unforgettable memories always remain with me."
stories = summarize_story(prompt, model, tokenizer, num_return_sequences=5)
print(stories)

There is a lot of talk that Dalat is the City of Eternal Spring, but I cannot understand why they would call it that.
There is a lot of talk that Dalat is the city of Eternal Spring, but I cannot understand why they would call it that.
There is a lot of talk that Dalat is the City of Eternal Spring, but I cannot understand why they would even contemplate leaving it.
There is a lot of talk that Dalat is the City of Eternal Spring, but I cannot understand why they would even contemplate moving it.
There is a lot of talk that Dalat is the City of Eternal Spring.
